In [41]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
import requests
import pandas as pd
from src.tables import get_machines
from src.preprocess import preprocess
from src.utils import df_na_vals

In [43]:
df = get_machines()
# preprocess(raw)

In [44]:
cpu_name = df.cpu_name.str.replace(r'^\d\dth Gen ','')
cpu_name = cpu_name.str.replace(r'^AMD','')
cpu_name = cpu_name.str.replace(r'Threadripper','TR')
cpu_name = cpu_name.str.replace(r'Processor$','')
cpu_name = cpu_name.str.replace(r'-Core','C')
cpu_name = cpu_name.str.replace(r' with Radeon Graphics','')
# cpu_name = df.cpu_name.str.replace('Core™', '')
df.cpu_name = cpu_name

df['cores'] = (df.cpu_cores / df.num_gpus).fillna(0).astype(int)
df['ram'] = (df.cpu_ram / df.num_gpus).round().astype(int)
df['dph'] = (df.dph_base / df.num_gpus).round().astype(int)
df['disk'] = (df.disk_space / df.num_gpus).astype(int)
df['rented_fraq'] = '[' + df.num_gpus_rented.astype(str) + "/" + df.num_gpus.astype(str) + ']'
# df['rented'] = df.num_gpus_rented > 0
df['rented'] = df.num_gpus_rented == df.num_gpus
# df['min_fraq'] = df.min_chunk/df.num_gpus

# df.dlperf = df.dlperf.round(1)
# df.dlperf_per_dphtotal = df.dlperf_per_dphtotal.round(1)
# df.disk_bw = df.disk_bw.round()
# df.duration = pd.to_timedelta(df.duration * 1e9).dt.days
# df.inet_down = df.inet_down.fillna(0).astype(int)
# df.inet_up = df.inet_up.fillna(0).astype(int)
# df.inet_up_cost = (df.inet_up_cost * 1000).fillna(0).astype(int)
# df.inet_down_cost = (df.inet_down_cost * 1000).fillna(0).astype(int)

# df['isp'] = df.location.apply(lambda x: x.get('isp', None) if not pd.isna(x) else None)

# df.end_date = (pd.to_datetime(df.end_date * 1e9) - pd.Timestamp.now()).dt.days
df.end_date = pd.to_datetime(df.end_date * 1e9)
# df.start_date = pd.to_datetime(df.start_date * 1e9)
# df.set_index('machine_id', inplace=True)

In [45]:
cols = ['gpu_name',  'rented_fraq', 'dph', 'credit_discount_max', 'cores', 'ram', 'disk', 'disk_bw',
         'min_chunk', 'country', 'storage_cost', 'reliability', 'inet_up', 'inet_down', 'inet_up_cost', 'inet_down_cost', 'cuda_max_good', 'driver_version',  'pci_gen', 'gpu_lanes', 'hosting_type',  'pcie_bw', 'cpu_name', 'mobo_name',  'country', 'isp']

df.query("dph >= 0.5 and gpu_name == 'RTX 4090'")[cols].sort_values(by='dph', ascending=False)

,gpu_name,rented_fraq,dph,credit_discount_max,cores,ram,disk,disk_bw,min_chunk,country,...,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country,isp
71,RTX 4090,[0/1],4000,400,24,31,840,2052,1,AT,...,12.2,535.113.01,4,16,0,247,AMD Ryzen 9 3900X 12C Processor,PRIME B550,AT,M247 Europe SRL
555,RTX 4090,[0/2],2000,400,24,126,495,1655,1,UA,...,12.0,525.125.06,4,16,0,236,AMD Ryzen TR 3960X 24C Processor,TRX40 PRO 10G,UA,Zahidna Multyservisna Merezha
1307,RTX 4090,[0/3],1000,0,5,21,496,2255,1,JP,...,12.0,525.89.02,4,8,0,128,AMD Ryzen 7 5800X 8C Processor,MEG X570 UNIFY,JP,Ntt Business Solutions Corporation
1301,RTX 4090,[0/1],1000,0,8,31,170,939,1,JP,...,12.2,535.113.01,3,16,0,122,Core™ i7-7700,Z270,JP,Ntt Business Solutions Corporation
594,RTX 4090,[1/4],1000,0,12,63,415,1959,1,JP,...,12.2,535.86.05,4,16,0,219,AMD EPYC 7402 24C Processor,MZ32,JP,Ntt Business Solutions Corporation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,RTX 4090,[1/1],195,400,16,31,680,5552,1,AU,...,12.2,535.113.01,4,16,0,197,AMD Ryzen 7 5800X3D 8C Processor,MEG B550 UNIFY,AU,Fuzenet Pty
18,RTX 4090,[0/1],190,0,48,252,3250,6300,1,MY,...,12.2,535.113.01,4,16,0,220,AMD EPYC 7402 24C Processor,H12SSL,MY,TM Net
17,RTX 4090,[0/1],190,0,48,220,3250,7071,1,MY,...,12.2,535.113.01,4,16,0,227,AMD EPYC 7402 24C Processor,H12SSL,MY,TM Net
113,RTX 4090,[1/1],190,0,48,252,3250,6642,1,MY,...,12.2,535.113.01,4,16,0,222,AMD EPYC 7402 24C Processor,H12SSL,MY,TM Net


In [48]:
df.query('num_gpus == 12')[cols]

,gpu_name,rented_fraq,dph,credit_discount_max,cores,ram,disk,disk_bw,min_chunk,country,...,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country,isp
279,RTX 4090,[8/12],700,400,21,42,129,1326,1,US,...,12.2,535.113.01,4,8,0,127,AMD EPYC 7742 64C Processor,ROME2D32GM,US,Cox Communications
459,RTX 4090,[12/12],500,200,10,42,376,5187,4,GB,...,12.2,535.86.05,4,8,0,127,AMD EPYC 7713 64C Processor,ROMED8,GB,Vodafone UK
589,RTX 4090,[12/12],700,400,21,42,181,1702,4,US,...,12.2,535.113.01,4,8,0,127,AMD EPYC 7742 64C Processor,ROME2D32GM,US,Cox Communications
849,RTX 4090,[4/12],700,400,21,42,203,2351,1,US,...,12.2,535.113.01,4,8,0,123,AMD EPYC 7742 64C Processor,ROME2D32GM,US,Cox Communications
850,RTX 4090,[2/12],600,400,21,42,10,3180,1,US,...,12.2,535.113.01,4,8,0,124,AMD EPYC 7742 64C Processor,ROME2D32GM,US,Cox Communications
891,RTX 4090,[12/12],750,400,21,42,222,2883,4,US,...,12.2,535.113.01,4,8,0,128,AMD EPYC 7742 64C Processor,ROME2D32GM,US,Cox Communications
952,RTX 3090,[12/12],290,400,1,42,66,3954,12,US,...,12.0,525.125.06,4,8,0,128,AMD EPYC 7F32 8C Processor,ROMED16QM3,US,Optimum Online
953,RTX 3090,[0/12],290,400,10,21,64,4067,12,US,...,12.0,525.125.06,4,8,0,128,AMD EPYC 7H12 64C Processor,ROMED16QM3,US,Optimum Online
954,RTX 3090,[0/12],290,400,10,42,557,1727,12,US,...,12.0,525.125.06,4,8,0,127,AMD EPYC 7H12 64C Processor,ROMED16QM3,US,Optimum Online
1255,RTX A4000,[8/12],100,400,1,3,68,434,4,CA,...,12.0,525.105.17,2,1,0,3,Core™ i9-9900K,TB360,CA,TELUS DSL Internet


In [47]:
df[cols].query('country == "RU"')

,gpu_name,rented_fraq,dph,credit_discount_max,cores,ram,disk,disk_bw,min_chunk,country,...,cuda_max_good,driver_version,pci_gen,gpu_lanes,hosting_type,pcie_bw,cpu_name,mobo_name,country,isp
56,RTX 4090,[0/1],330,50,36,126,3350,3220,1,RU,...,12.2,535.113.01,3,16,0,117,Xeon® E5-2696 v3,X99,RU,Rostelecom
57,RTX 4090,[0/1],330,50,36,126,3350,3265,1,RU,...,12.2,535.113.01,3,16,0,117,Xeon® E5-2696 v3,X99,RU,Rostelecom
89,RTX 3090,[1/1],170,400,16,63,810,3978,1,RU,...,12.2,535.113.01,4,16,0,245,AMD Ryzen 7 5800 8C Processor,3716,RU,TransTeleCom
126,RTX 3080 Ti,[0/2],146,50,18,63,1675,3341,1,RU,...,12.2,535.113.01,3,16,0,118,Xeon® E5-2696 v3,X99,RU,Rostelecom
129,RTX 3090,[0/8],450,400,12,63,796,3403,8,RU,...,12.2,535.113.01,4,16,0,241,AMD EPYC 7402 24C Processor,H12DSG,RU,ITSOFT LLC
217,RTX 3090,[1/1],307,200,128,252,810,6613,1,RU,...,12.2,535.104.12,4,16,0,224,AMD EPYC 7742 64C Processor,ROMED8,RU,Netorn LLC
414,RTX 4090,[0/1],900,400,128,252,1590,3570,1,RU,...,12.2,535.86.05,4,16,0,238,AMD EPYC 7742 64C Processor,H12SSL,RU,Rostelecom
433,RTX 3080 Ti,[3/3],140,50,10,42,1076,5894,1,RU,...,12.2,535.104.05,4,4,0,61,AMD Ryzen 9 5950X 16C Processor,ROG STRIX X570,RU,Rostelecom
586,RTX 3080 Ti,[2/2],140,50,18,63,1630,3337,1,RU,...,12.2,535.113.01,3,16,0,118,Xeon® E5-2696 v3,X99,RU,Rostelecom
632,RTX 3080,[6/6],73,400,2,2,50,360,1,RU,...,12.0,525.125.06,1,1,0,1,Core™ i5-10400F,H510 Pro BTC,RU,Iskratelecom CJSC
